In [1]:
import os

import graph_nets as gns
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from bclassification.utils_base import (
    print_class_weights,
    compute_weight_bias,
    print_dataset,
)
from bclassification.utils_gnsn import create_dataset
from experience import load_experience
from lib.constants import Constants as Const
from lib.data_utils import (
    make_dir,
    env_pf,
    create_results_dir,
)
from lib.gns import get_graph_feature_dimensions
from lib.visualizer import Visualizer, pprint

Visualizer()

experience_dir = make_dir(os.path.join(Const.EXPERIENCE_DIR, "data-aug"))
# experience_dir = make_dir(os.path.join(Const.RESULTS_DIR, "performance-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "_bc-gnsn"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))
case, collector = load_experience(case_name, agent_name, experience_dir, env_dc=env_dc)


L2RPN_2019_ART (dc)


--------------------------------------------------------------------------------
                                        Loading Experience
--------------------------------------------------------------------------------
    - Loading chronics:                 ./experience/data-aug/l2rpn_2019_art-dc/agent-mip-chronic-****
    - Number of loaded chronics:        4


In [5]:
"""
    Parameters
"""
random_seed = 0

mode = "binary"

n_window_targets = 0
n_window_history = 1
downsampling_rate = 0.1
feature_scaling = True
val_frac = 0.10

# Model
model_type = "gn"
dropout_rate = 0.0
l2_reg = 0.0
n_hidden = 256
n_hidden_layers = 4
threshold = 0.50

# Training
learning_rate = 1e-5
n_batch = 512
n_epochs = 1000

In [39]:
import numpy as np

from lib.action_space import is_do_nothing_action
from lib.data_utils import (
    moving_window,
    extract_history_windows,
)
from lib.dc_opf import TopologyConverter
from lib.visualizer import pprint


def obs_to_vects(obs, tc, mode):
    lines_or_to_sub_bus = tc.lines_or_to_sub_bus(obs)
    lines_ex_to_sub_bus = tc.lines_ex_to_sub_bus(obs)
    gens_to_sub_bus = tc.gens_to_sub_bus(obs)
    loads_to_sub_bus = tc.loads_to_sub_bus(obs)
    
    gen_to_sub_id = tc.gen_to_sub_id
    load_to_sub_id = tc.load_to_sub_id
    line_or_to_sub_id = tc.line_or_to_sub_id
    line_ex_to_sub_id = tc.line_ex_to_sub_id
    
    if mode == "binary":
        line_vect = list()
        line_vect.append(np.atleast_2d(obs.p_or).T)
        line_vect.append(np.atleast_2d(obs.rho).T)
        for bus in [1, 2]:
            mask_or = np.equal(lines_or_to_sub_bus, bus).astype(np.float)
            mask_ex = np.equal(lines_ex_to_sub_bus, bus).astype(np.float)
            mask_or = np.multiply(mask_or, obs.line_status.astype(np.float))
            mask_ex = np.multiply(mask_ex, obs.line_status.astype(np.float))
            
            line_vect.append(np.atleast_2d(mask_or).T)
            line_vect.append(np.atleast_2d(mask_ex).T)
        
        line_vect.append(np.atleast_2d(obs.time_before_cooldown_line.astype(np.float)).T)
        line_vect.append(np.atleast_2d(obs.timestep_overflow.astype(np.float)).T)
        # line_vect.append(np.atleast_2d(obs.time_next_maintenance.astype(np.float)).T)
        # line_vect.append(np.atleast_2d(obs.duration_next_maintenance.astype(np.float)).T)
        line_vect = np.nan_to_num(np.hstack(line_vect))
        
        sub_vect = list()
        for sub_id in range(tc.n_sub):
            vect = list()

            mask_sub_gen = np.equal(gen_to_sub_id, sub_id).astype(np.float)
            mask_sub_load = np.equal(load_to_sub_id, sub_id).astype(np.float)
            
            prods = np.multiply(mask_sub_gen, obs.prod_p)
            loads = np.multiply(mask_sub_load, obs.load_p)            
            
            vect.append(prods)
            vect.append(loads)
            for bus in [1, 2]:
                mask_bus_gen = np.equal(gens_to_sub_bus, bus).astype(np.float)
                mask_gen = np.multiply(mask_bus_gen, mask_sub_gen)
                
                mask_bus_load = np.equal(loads_to_sub_bus, bus).astype(np.float)
                mask_load = np.multiply(mask_bus_load, mask_sub_load)

                vect.append(mask_gen)
                vect.append(mask_load)
                
            vect = np.hstack(vect)
            sub_vect.append(vect)
                
        sub_vect = np.nan_to_num(np.vstack(sub_vect))
                
        vects = (
            line_vect.astype(np.float),
            sub_vect.astype(np.float),
        )
    elif mode == "structured":
        line_vect = list()
        line_vect.append(np.atleast_2d(obs.p_or).T)
        line_vect.append(np.atleast_2d(obs.rho).T)
        for bus in [1, 2]:
            mask_or = np.equal(lines_or_to_sub_bus, bus).astype(np.float)
            mask_ex = np.equal(lines_ex_to_sub_bus, bus).astype(np.float)
            mask_or = np.multiply(mask_or, obs.line_status.astype(np.float))
            mask_ex = np.multiply(mask_ex, obs.line_status.astype(np.float))
            
            line_vect.append(np.atleast_2d(mask_or).T)
            line_vect.append(np.atleast_2d(mask_ex).T)
        
        line_vect.append(np.atleast_2d(obs.time_before_cooldown_line.astype(np.float)).T)
        line_vect.append(np.atleast_2d(obs.timestep_overflow.astype(np.float)).T)
        # line_vect.append(np.atleast_2d(obs.time_next_maintenance.astype(np.float)).T)
        # line_vect.append(np.atleast_2d(obs.duration_next_maintenance.astype(np.float)).T)
        line_vect = np.nan_to_num(np.hstack(line_vect))
        
        sub_vect = list()
        for sub_id in range(tc.n_sub):
            mask_sub_gen = np.equal(gen_to_sub_id, sub_id).astype(np.float)
            mask_sub_load = np.equal(load_to_sub_id, sub_id).astype(np.float)
            
            vect = list()
            for bus in [1, 2]:
                mask_bus_gen = np.equal(gens_to_sub_bus, bus).astype(np.float)
                mask_gen = np.multiply(mask_bus_gen, mask_sub_gen)
                prods = np.multiply(mask_gen, obs.prod_p)
                
                mask_bus_load = np.equal(loads_to_sub_bus, bus).astype(np.float)
                mask_load = np.multiply(mask_bus_load, mask_sub_load)
                loads = np.multiply(mask_load, obs.load_p)

                vect.append(prods)
                vect.append(loads)
                
            vect = np.hstack(vect)
            sub_vect.append(vect)
                
        sub_vect = np.nan_to_num(np.vstack(sub_vect))
                
        vects = (
            line_vect.astype(np.float),
            sub_vect.astype(np.float),
        )
    else:
        raise ValueError("Invalid mode.")

    return vects


def create_dataset(
    case,
    collector,
    mode,
    n_window_targets=0,
    n_window_history=0,
    downsampling_rate=1.0,
    feature_scaling=True,
):
    tc = TopologyConverter(case.env)
    process_fn = lambda obs: obs_to_vects(obs, tc, mode)
    combine_fn = lambda obs_vects: np.concatenate(obs_vects)

    X_all = []
    Y_all = []
    mask_targets = []

    obs_sample = None

    pprint("    - Input structure:", mode)

    for chronic_idx, chronic_data in collector.data.items():
        chronic_len = len(chronic_data["actions"])
        chronic_obses = chronic_data["obses"][:chronic_len]
        chronic_labels = is_do_nothing_action(
            chronic_data["actions"][:chronic_len], case.env, dtype=np.bool
        )

        # Mask
        mask_positives = extract_history_windows(
            chronic_labels, n_window=n_window_targets
        )
        mask_negatives = np.logical_and(
            np.random.binomial(1, downsampling_rate, len(chronic_labels)).astype(
                np.bool
            ),
            ~mask_positives,
        )
        chronic_mask_targets = np.logical_or(chronic_labels, mask_negatives)

        # Observation
        obs_sample = chronic_obses[0]
        chronic_X = moving_window(
            chronic_obses,
            n_window=n_window_history,
            process_fn=process_fn,
            combine_fn=combine_fn,
            padding=np.zeros_like(process_fn(obs_sample),
        )
        chronic_X = np.vstack(chronic_X)

        X_all.append(chronic_X)
        Y_all.extend(chronic_labels.astype(np.float))
        mask_targets.extend(chronic_mask_targets)

    X_all = np.vstack(X_all)
    Y_all = np.array(Y_all)
    mask_targets = np.array(mask_targets)
        
create_dataset(
    case,
    collector,
    mode=mode,
    n_window_history=n_window_history,
    n_window_targets=n_window_targets,
    downsampling_rate=downsampling_rate,
    feature_scaling=feature_scaling,
)

print("gen", case.env.n_gen)
print("load", case.env.n_load)
print("line", case.env.n_line)

    - Input structure:                  binary
(20, 8)
[[ 66   0   1   1   0   0   0   0]
 [ 49   1   1   1   0   0   0   0]
 [ 30   0   1   1   0   0   0   0]
 [ 49   1   1   1   0   0   0   0]
 [ 41   1   1   1   0   0   0   0]
 [ 16   0   1   1   0   0   0   0]
 [-39   1   1   1   0   0   0   0]
 [ 32   1   1   1   0   0   0   0]
 [ 18   1   1   1   0   0   0   0]
 [ 43   1   1   1   0   0   0   0]
 [ 18   1   1   1   0   0   0   0]
 [  8   0   1   1   0   0   0   0]
 [  6   0   1   1   0   0   0   0]
 [  0   0   1   1   0   0   0   0]
 [ 32   1   1   1   0   0   0   0]
 [ 11   0   1   1   0   0   0   0]
 [  7   0   1   1   0   0   0   0]
 [ -2   0   1   1   0   0   0   0]
 [  1   0   1   1   0   0   0   0]
 [  5   0   1   1   0   0   0   0]]
(160,)
(14, 48)
[[  0   0   0   0 118   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]
 [ 77   0   0   0   0  24   0

In [ ]:
"""
    Dataset
"""

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

X, Y, mask_targets, X_all, Y_all = create_dataset(
    case,
    collector,
    mode=mode,
    n_window_history=n_window_history,
    n_window_targets=n_window_targets,
    downsampling_rate=downsampling_rate,
    feature_scaling=feature_scaling,
)

class_weight, initial_bias = compute_weight_bias(Y)

X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=val_frac, random_state=random_seed
)

mask_test_neg = np.logical_and(
    ~mask_targets, np.random.binomial(1, val_frac, mask_targets.size).astype(np.bool)
)
X_test = np.concatenate((X_val, X_all[mask_test_neg, :]))
Y_test = np.concatenate((Y_val, Y_all[mask_test_neg]))

print_dataset(X_all, Y_all, "All data")
print_dataset(X, Y, "Data")
print_dataset(X_train, Y_train, "Train")
print_dataset(X_val, Y_val, "Validation")
print_dataset(X_test, Y_test, "Test")
print_class_weights(class_weight)
pprint("Initial bias:", "{:.4f}".format(float(initial_bias)))

model_dir = create_results_dir(case_results_dir, model_name=model_type)